In [1]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd

## input variable

In [2]:
View_Column_name = "PrevContext_NextContext_PrevTarget_NextTarget_ShapeTarget"
DATE = input()
DATE

'2021-01-14/22-31-43/projector/in_test/199'

## load data

In [3]:
# load csv
df = pd.read_csv("../outputs/{}/tensor.csv".format(DATE), header=None, delimiter="\t")
df_label = pd.read_csv("../outputs/{}/metadata.tsv".format(DATE), delimiter="\t", index_col=0)

In [4]:
data = pd.concat([df_label, df], axis=1)
del df, df_label

## def loss

In [7]:
import scipy.spatial.distance
def cossim(x, y):
    return scipy.spatial.distance.cosine(x, y)

def mean_cossim(X, Y):
    tmp = 0.
    for i in range(X.shape[0]):
        x = X[i, :]
        y = Y[i, :]
        tmp += cossim(x, y)
    return 1-tmp/X.shape[0]

## select vector

In [8]:
vector_dict = {
    "prev_context": None,
    "next_context": None,
    "prev_target": None,
    "next_target": None,
    "now_target": None,
}
for i, name in enumerate(vector_dict.keys()):
     vector_dict[name] = np.array(data.loc[data[View_Column_name]==i].iloc[:, 2:])

## Calculate

In [9]:
tmp = []
for i, (k, v) in enumerate(vector_dict.items()):
    for j, (k_, v_) in enumerate(vector_dict.items()):
        if i<j and k != k_:
            cossim_value = mean_cossim(v, v_)
            print("{} VS {} \t\t {:.10f}".format(k, k_, cossim_value))
            tmp.append([k, k_, cossim_value, 1-cossim_value])

prev_context VS next_context 		 0.9565499024
prev_context VS prev_target 		 0.9994714011
prev_context VS next_target 		 0.9626253615
prev_context VS now_target 		 0.9825647002
next_context VS prev_target 		 0.9592547946
next_context VS next_target 		 0.9991202646
next_context VS now_target 		 0.9769976896
prev_target VS next_target 		 0.9649074388
prev_target VS now_target 		 0.9835774295
next_target VS now_target 		 0.9803349126


In [10]:
# 20k samples
df = pd.DataFrame(tmp, columns=["vector A", "vector B", "cossim", "cossim_loss"])
df.sort_values("cossim", ascending=False)

,vector A,vector B,cossim,cossim_loss
1,prev_context,prev_target,0.999471,0.000529
5,next_context,next_target,0.999120,0.000880
8,prev_target,now_target,0.983577,0.016423
3,prev_context,now_target,0.982565,0.017435
9,next_target,now_target,0.980335,0.019665
6,next_context,now_target,0.976998,0.023002
7,prev_target,next_target,0.964907,0.035093
2,prev_context,next_target,0.962625,0.037375
4,next_context,prev_target,0.959255,0.040745
0,prev_context,next_context,0.956550,0.043450


## shape(t)同士の距離

In [13]:
vector_withlabel_dict = {
    "prev_context": {
        "square_circle": None,
        "triangle_circle": None,
        "square_triangle": None
    },
    "next_context": {
        "square_circle": None,
        "triangle_circle": None,
        "square_triangle": None
    },
    "prev_target": {
        "square_circle": None,
        "triangle_circle": None,
        "square_triangle": None
    },
    "next_target": {
        "square_circle": None,
        "triangle_circle": None,
        "square_triangle": None
    },
    "now_target": {
        "square_circle": None,
        "triangle_circle": None,
        "square_triangle": None
    },
}
for i, (name, label_dict) in enumerate(vector_withlabel_dict.items()):
    for j, label in enumerate(label_dict.keys()):
         vector_withlabel_dict[name][label] = np.array(data.loc[data[View_Column_name]==i].loc[data["Label"]==j].iloc[:, 2:])

In [50]:
tmp = []
def split_name(prev_next_label, shape_label):
    prev_next = prev_next_label.split("_")[0]
    prev_, next_ = shape_label.split("_")
    if prev_next == "prev":
        return prev_
    elif prev_next == "next":
        return next_
for i, (key, label_dict) in enumerate(vector_withlabel_dict.items()):
    for j, (label, v) in enumerate(label_dict.items()):
        for k, (key_, label_dict_) in enumerate(vector_withlabel_dict.items()):
            for l, (label_, v_) in enumerate(label_dict_.items()):
                if "context" in key and "context" in key_:
                    #if key!=key_ and label!=label_:
                    if i<=k:
                        cossim_value = mean_cossim(v, v_)
                        ashape = split_name(key, label)
                        bshape = split_name(key_, label_)
                        tmp.append([
                            key, label, key_, label_,
                            ashape+" vs "+bshape, cossim_value, 1-cossim_value
                            ])


In [52]:
df = pd.DataFrame(tmp, columns=["A", "Ashape", "B", "Bshape", "A_B", "cossim", "cossim_loss"])
df.groupby("A_B").mean().sort_values("cossim", ascending=False)

,cossim,cossim_loss
A_B,,
triangle vs triangle,0.999778,0.000222
square vs square,0.999685,0.000315
circle vs circle,0.999591,0.000409
square vs circle,0.970947,0.029053
triangle vs square,0.952149,0.047851
square vs triangle,0.951924,0.048076
triangle vs circle,0.946423,0.053577
circle vs triangle,0.946141,0.053859
